# Visualizing Geospatial Point Data with Python : Meteorite Landings 

This notenook shows how to use several open source tools and techniques for visualizing points data on differents types of maps.

For this, we will use :
- [GeoPandas](https://geopandas.org/en/stable/) to store data like [Pandas](https://pandas.pydata.org/) but with spatial information and operations
- [BaseMap](https://matplotlib.org/basemap/stable/) and [Geoplot](https://residentmario.github.io/geoplot/), two extentions of matplotlib
- [Folium](https://python-visualization.github.io/folium/latest/) to plot interactive maps
- [Cartopy](https://scitools.org.uk/cartopy/docs/latest/index.html) to add satellites images

## Database initialisation

This data set from The Meteoritical Society contains information on all of the known meteorite landings. They were collected by Javier de la Torre and consist of 34,513 meteorites including the following fields like longitude, latitude, mass, year, ...

The link to the dataset: https://data.nasa.gov/Space-Science/Meteorite-Landings/gh4g-9sfh/about_data

The link of the S2 catalogue used at the end: https://catalogue.theia-land.fr/

In [ ]:
# import library
import numpy as np                       # linear algebra
import pandas as pd                      # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt          # plotting library

plt.ion();

In [ ]:
path_data = "data/Meteorite_Landings_20240710.csv"

# Create a DataFrame from the CSV file
df = pd.read_csv(path_data)

# Firstly, let's clean the dataset and keep valid entries with known latitude and longitude 
# Plus, 0N/0E locations need to be treated as nan values
df = df.dropna(subset = ['reclong', 'reclat', "mass (g)", "year"])
df = df.drop(df.loc[(df.reclat == 0.0) & (df.reclong == 0.0)].index)

# Display the first 5 values
df.head()

## Plots examples

### Basemap

Lets's try some plots using Basemap, a simple matplotlib library for plotting 2D data on maps.

#### Meteorites Impacts on simple map

In [ ]:
from mpl_toolkits.basemap import Basemap 

plt.figure(figsize=(20, 10))

map = Basemap(projection='cyl')
map.drawmapboundary(fill_color='w')
map.drawcoastlines(linewidth=0.5)
map.drawmeridians(range(0, 360, 20), linewidth=0.1)

# equator and tropics and polar circles
map.drawparallels([-66.56083,-23.5,0.0,23.5,66.56083], linewidth=0.6)

map.scatter(df.reclong, df.reclat, marker='.',alpha=0.25, c='red',edgecolor='None')
plt.title('Map of all impacts of meteorites', fontsize=10);

#### Meteorites Impacts with their mass of colored map

In [ ]:
plt.figure(figsize=(20, 10))

map = Basemap(projection='cyl')
map.etopo() # add an etopo relief image as map background
map.drawcountries() # add country boundaries
map.drawmeridians(range(0, 360, 20),linewidth=0.1)
map.drawparallels([-66.56083,-23.5,0.0,23.5,66.56083], linewidth=0.6)

# Add impacts with markers size proportional to their mass
map.scatter(df.reclong,df.reclat,s=np.sqrt(df["mass (g)"]/300),alpha=1.0,color='r',edgecolors='k')
plt.title('Map of all impacts of meteorites with their mass', fontsize=10);

### GeoPandas

GeoPandas is library that adds geographic data to pandas objects. All we need is a new "geometric" column with all the points (in this case).

In [ ]:
import geopandas as gpd

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.reclong, df.reclat), crs="EPSG:4326"
)

# Display the first 5 values
gdf.head()

#### Worlds countries

We create a new Geopandas from an existing Shapefile with the polygonal description of all countries and count how many meteorites has landed into. 

In [ ]:
# Load polygons
world = gpd.read_file("data/world.shp")
 
# Itération par départements
monde_regions_meteor = []
for index, row in world.iterrows() :
    monde_regions_meteor.append(gdf.within(row["geometry"]).sum())
world["nb_meteors"] = monde_regions_meteor

#### Meteorites Impacts with simple map

Each Geopandas plot includes by default a map.

In [ ]:
gdf.plot(ax=world.plot(figsize=(20, 10), color='white', edgecolor='black'), 
         color='red', marker='x', markersize=2).set_title("Map of all impacts");
# add markersize=column for proportional marker

#### Number of meteorites landings by french departement

We create a new Geopandas from an existing Shapefile with the polygonal description of all departments and count how many meteorites has landed into.

In [ ]:
# Load polygons
metropole_departements = gpd.read_file("data/departements_metro.shp")

# Iteration by departments
# Can be a little long to compute
counts_by_departements = []
for index, row in metropole_departements.iterrows() :
    counts_by_departements.append(gdf.within(row["geometry"]).sum())
metropole_departements["nb_meteors"] = counts_by_departements

#### Repartition of landing by French Departments 

In [ ]:
metropole_departements.plot(figsize=(10, 9), column="nb_meteors", legend=True, 
                            legend_kwds={"label": "Number of meteorites impacts by french department", "orientation": "horizontal"});

#### Number of meteorites impacts by french department with one and more landing(s)

In [ ]:
metropole_departements_positive = metropole_departements.copy()
metropole_departements_positive.replace(0, np.nan, inplace=True)

metropole_departements_positive.plot(figsize=(10, 9), column="nb_meteors", legend=True, 
                      legend_kwds={"label": "Number of meteorites impacts by french department", "orientation": "horizontal"}, 
                      missing_kwds={ "color": "lightgrey", "edgecolor": "black", "hatch": "///", "label": "No landings"});

## Folium 

Folium is one of the many Python libraries used for visualizing geographic databases via interactive maps.

In [ ]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

#### Meteorites Impacts by clusters

In [ ]:
# Create empty map with OpenStreetMap centered on Toulouse
map_center_toulouse = folium.Map(location=[43.600000, 1.433333], tiles="OpenStreetMap", zoom_start=3)

# Add points 
mc = MarkerCluster()
for idx, row in gdf.iterrows():
    mc.add_child(Marker([row.reclat, row.reclong], popup=str(row["mass (g)"]) + " kg"))
map_center_toulouse.add_child(mc)

# To do the simpliest map :
# folium.GeoJson(gdf).add_to(map_center_toulouse)   

map_center_toulouse

#### Meteorites Impacts by heatmap

The interactive heatmap proposed by the library can be very interesting to help understand the distribution of data, in this case the areas where the meteorites have the most crashed.

In [ ]:
# Create empty map with CartoDB
map = folium.Map(location=[0.0, 0.0], tiles='cartodbpositron', zoom_start=2)

# Create heatmap
HeatMap(data=gdf[['reclat', 'reclong']], radius=10).add_to(map)
map

#### Number of meteorites impacts by french department

In [ ]:
# Create an empty map
map_center_toulouse = folium.Map(location=[43.600000, 1.433333], tiles="OpenStreetMap", zoom_start=5)

# Create the choropleth map
chloropleth = folium.Choropleth(geo_data = metropole_departements,
                                name="choropleth",
                                data = metropole_departements,
                                columns=["nom", "nb_meteors"],
                                key_on='feature.properties.nom',
                                fill_color = 'YlGnBu_r',
                                fill_opacity = 0.5,
                                line_opacity = 1,
                                bins=list(set(metropole_departements["nb_meteors"].values)),
                                legend_name = 'Number of meteorites impacts by french department')

# Crate and add legend
chloropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['nom'],labels=True))

chloropleth.add_to(map_center_toulouse)

map_center_toulouse

## GeoPlot

It is an other extention of matplotlib with some additional tools. 


#### Number of meteorites impacts by country by cluster values

This functionality of grouping by class can be interesting when, as our case cointains some extreme values.

In [ ]:
import geoplot as gplt
import mapclassify as mc
import geoplot.crs as gcrs

scheme = mc.FisherJenks(world['nb_meteors'], k=8)
gplt.choropleth(
    world, hue='nb_meteors', projection=gcrs.PlateCarree(),
    edgecolor='black', linewidth=1,
    cmap='Greens',
    legend=True, legend_kwargs={'loc': 'lower left'},
    scheme=scheme,
    figsize=(20, 10)
    #legend_labels=[...]
)
plt.title("Number of Impacts by Country");

### Cartopy

Cartopy is another Python package designed for geospatial data processing in order to produce maps and other geospatial data analyses.

In [ ]:
from osgeo import gdal, osr

gdal.UseExceptions()

"""
# We previously took a S2 image and resampled it to be lighter to work with
image = 'data/S2A_MSIL1C_20240708T105031_N0510_R051_T31TCJ_20240708T125024.SAFE/GRANULE/L1C_T31TCJ_A047238_20240708T105028/IMG_DATA/T31TCJ_20240708T105031_B04.jp2'
ds = gdal.Open(image)
data = ds.ReadAsArray()
gt = ds.GetGeoTransform()
options = gdal.WarpOptions(xRes= gt[1]*20, yRes= gt[5] * 20)
image_res = 'data/tmp/T31TCJ_20240708T105031_B04_resampled.jp2'
gdal.Warp(image_res, iname, options=options)
"""

image_res = 'data/T31TCJ_20240708T105031_B04_resampled.jp2'

In [ ]:
from pyproj import Transformer

import rasterio

dataset = rasterio.open(image_res)

# Get image information
epsg = dataset.crs

img_extent = [dataset.bounds[0], dataset.bounds[2], dataset.bounds[1], dataset.bounds[3]] 
print("Lon min:", img_extent[0], "- max:", img_extent[1])
print("Lat min:", img_extent[2], "- max:", img_extent[3])
transformer = Transformer.from_crs(epsg, 4326, always_xy=True)

img_extent_lat_lon = transformer.transform(img_extent[0], img_extent[2]) + transformer.transform(img_extent[1], img_extent[3])
img_extent_lat_lon = [img_extent_lat_lon[0], img_extent_lat_lon[2], img_extent_lat_lon[1], img_extent_lat_lon[3]]
print("Lon min:", img_extent_lat_lon[0], "- max:", img_extent_lat_lon[1])
print("Lat min:", img_extent_lat_lon[2], "- max:", img_extent_lat_lon[3])

In [ ]:
# Select impacts in the image geometry
gdf_s2 = gdf.loc[(img_extent_lat_lon[0] < gdf["reclong"]) & (gdf["reclong"]<  img_extent_lat_lon[1])]
gdf_s2 = gdf_s2.loc[( img_extent_lat_lon[2] < gdf_s2["reclat"] ) & (gdf_s2["reclat"] <  img_extent_lat_lon[3])]
gdf_s2.describe()

#### Plot impacts around Toulouse into T31TCJ S2 satellite image

In [ ]:
import cartopy.crs as ccrs

# Read the image and add a threshold to the image to be visually comprehensive
img = plt.imread(image_res)
img = np.array(img)
img[img > 4000] = 4000 # change plot dynamique

# Creation of the plot   
fig = plt.figure(figsize=(8, 12))

# Make the map
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_title("Impacts of meteorites in T31TCJ S2 tile")

# Add the image
ax.imshow(img, origin='upper', extent=img_extent_lat_lon, transform=ccrs.PlateCarree(), cmap="gray")

print (gdf_s2)
# Mark landing impacts
ax.plot(gdf_s2["reclong"], gdf_s2["reclat"], 'rx', markersize=4, transform=ccrs.Geodetic())
for idx, row in gdf_s2.iterrows(): 
    plt.text(row['reclong'] - 0.1, row['reclat'] - 0.03, int(row['year']))

# Add some cities
ax.text(1.433333, 43.600000, 'Toulouse', transform=ccrs.Geodetic(), bbox=dict(facecolor='white', alpha=0.5, boxstyle='round'))
ax.text(1.35, 44.0167, 'Montauban', transform=ccrs.Geodetic(), bbox=dict(facecolor='white', alpha=0.5, boxstyle='round'))
ax.text(0.633333, 44.200000, 'Agen', transform=ccrs.Geodetic(), bbox=dict(facecolor='white', alpha=0.5, boxstyle='round'))

# Add coordinates legends
gl = ax.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False

plt.show()

#### Plot impacts in south of France with T31TCJ S2 satellite image

In [ ]:
from shapely.geometry.polygon import Polygon
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Creation of the plot   
plt.figure(figsize=(8, 12))

# Make the map
bounds = [ -1.0, 5.0, 42.0, 45.0]
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_title("Impacts of meteorites in South of France")

ax.set_extent(bounds, crs=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE.with_scale('110m'), linewidth=0.75)

# Mark impacts
ax.plot(gdf["reclong"], gdf["reclat"], 'kx', markersize=4, transform=ccrs.Geodetic())

# Add departments geometry
ax.add_geometries(metropole_departements["geometry"].values, crs=ccrs.PlateCarree(), facecolor='none', edgecolor='red', alpha=0.5)

# Add image
ax.imshow(img, origin='upper', extent=img_extent_lat_lon, transform=ccrs.PlateCarree(), cmap="gray")

plt.show()